### CPSC 302 - Assignment 3
Shubhaankar Sharma (88601356)


#### Question 1

In [37]:
function gaussian_elimination(A::Matrix, b::Vector)
    n = size(A)[1]
    for k = 1:n-1
        for i = k+1:n
            factor = A[i,k]/A[k,k]
            A[i,k] = 0
            for j = k+1:n
                A[i,j] = A[i,j] - factor*A[k,j]
            end
            b[i] = b[i] - factor*b[k]
        end
    end

    return A, b
end

A = Float64[10 -1 2 0 3; -1 11 -1 3 4; 2 -1 10 -1 1; 0 3 -1 8 2; 3 4 1 2 12]
b = Float64[0, 0, 0, 0, 0]
display(A)
A_upper, b_new = gaussian_elimination(A, b)
display(A_upper)

5×5 Matrix{Float64}:
 10.0  -1.0   2.0   0.0   3.0
 -1.0  11.0  -1.0   3.0   4.0
  2.0  -1.0  10.0  -1.0   1.0
  0.0   3.0  -1.0   8.0   2.0
  3.0   4.0   1.0   2.0  12.0

5×5 Matrix{Float64}:
 10.0  -1.0   2.0       0.0       3.0
  0.0  10.9  -0.8       3.0       4.3
  0.0   0.0   9.54128  -0.779817  0.715596
  0.0   0.0   0.0       7.11058   0.875
  0.0   0.0   0.0       0.0       9.24233

### Question 2 b

In [50]:
import LinearAlgebra;

A = Float64[1 -2 1; 5 -1 -5;2 -4 4]
l = LinearAlgebra.lu(A)
display(l.L)
display(l.U)
display(l.p)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.4  1.0  0.0
 0.2  0.5  1.0

3×3 Matrix{Float64}:
 5.0  -1.0  -5.0
 0.0  -3.6   6.0
 0.0   0.0  -1.0

3-element Vector{Int64}:
 2
 3
 1

### Question 3

(a) Julia Code to generate 1,000,000 tridiagonal matrix
```julia
using LinearAlgebra

n = 1000000
m = diagm(-1 => rand(n-1),
           0 => rand(n),
           1 => rand(n-1))
```
Output on terminal: `[1]  + 5786 killed     julia`

(d)

In [19]:
function solve_tri(b, main_diag, super_diag, sub_diag)
    n = length(b)
    x = zeros(n)
    p = collect(1:n)
    b_ = copy(b)

    # U
    md = copy(main_diag)      # main diagonal
    sd = copy(super_diag)     # super diagonal
    ssd = zeros(n-2)          # super super diagonal (created by pivoting)

    # for L
    multipliers = zeros(n-1)   # elimination multipliers

    for i in 1:(n-1)
        # pivot if element below is larger
        if abs(sub_diag[i]) > abs(md[i])
            # swap permutation rows and solution rows
            p[i], p[i+1] = p[i+1], p[i]
            b_[i], b_[i+1] = b_[i+1], b_[i]

            # in triangular matrix, there are only three elements in a row

            # swap main diagonal and sub diagonal
            md[i], sub_diag[i] = sub_diag[i], md[i]
            # swap super diagonal and main diagonal
            md[i+1], sd[i] = sd[i], md[i+1]
            # swap super super diagonal and super diagonal
            if i < n-1
                sd[i+1], ssd[i] = ssd[i], sd[i+1]
            end
        end

        if md[i] != 0
            multiplier = sub_diag[i] / md[i]
            multipliers[i] = multiplier

            # r_(i+1) = r_(i+1) - multiplier * r_i
            md[i+1] = md[i+1] - multiplier * sd[i]
            if i < n-1
                sd[i+1] = sd[i+1] - multiplier * ssd[i]
            end
            b_[i+1] = b_[i+1] - multiplier * b_[i]
        end
    end

    # backward substitution
    x[n] = b_[n] / md[n]
    for i in (n-1):-1:1
        sum = b_[i]
        if i < n
            sum -= sd[i] * x[i+1]
        end
        if i < n-1
            sum -= ssd[i] * x[i+2]
        end
        x[i] = sum / md[i]
    end

    return x
end

solve_tri (generic function with 1 method)

In [20]:
using LinearAlgebra

function generate_diags(n)
    # Create diagonals for our tridiagonal matrix
    main_diag = rand(n)
    super_diag = rand(n-1)
    sub_diag = rand(n-1)
    
    A = diagm(
        -1 => sub_diag,
        0 => main_diag,
        1 => super_diag)
    
    b = rand(n)

    return A, b, main_diag, super_diag, sub_diag
end


# Test 10x10
println("\nTesting 10x10 system:")
n = 10
A, b, md, sd, subd = create_test_system(n)

x1 = solve_tri(b, md, sd, subd)

x2 = A \ b

println("Difference: ", norm(x1 - x2))

# Test 100x100
println("\nTesting 100x100 system:")
n = 100
A, b, md, sd, subd = create_test_system(n)

x1 = solve_tri(b, md, sd, subd)

x2 = A \ b

println("Difference: ", norm(x1 - x2))


Testing 10x10 system:
Difference: 0.0

Testing 100x100 system:
Difference: 0.0
